제 1유형

1. 각 연도 및 성별의 총 대출액의 절댓값 차이를 구하고, 절댓값 차이가 가장 큰 지역 코드를 구하시오

2. 각 연도별 최대 검거율을 가진 범죄유형을 찾아서 해당 연도 및 유형의 검거건수들의 총합을 구하시오.

3. 제시된 문제를 순서대로 풀고, 해답을 제시하시오
    3.1. 평균만족도: 결측치는 평균만족도 컬럼의 전체 평균으로 채우시오.
    3.2. 근속연수: 결측치는 각 부서와 등급별 평균 근속연수로 채우시오.
    3.3. A: 부서가 'HR'이고 등급이 'A'인 사람들의 평균 근속연수를 계산하시오.
    3.4. B: 부서가 'Sales'이고 등급이 'B'인 사람들의 평균 교육참가횟수를 계산하시오.
    3.5. A와 B를 더한 값을 구하시오.

In [ ]:
import pandas as pd
df = pd.read_csv('9_1_1.csv')
#print(df.head())

q1 = df.groupby(['year', 'gender', '지역코드'])[['금액1', '금액2']].sum().reset_index()
q1['diff'] = abs(q1['금액1'] - q1['금액2'])
max_diff = q1.loc[q1['diff'].idxmax()]
print(max_diff['지역코드'])


167


In [35]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('9_1_2.csv')

# '발생건수'와 '검거건수' 데이터 분리
df_occur = df[df['구분'] == '발생건수']
df_arrest = df[df['구분'] == '검거건수']

# 데이터를 연도와 범죄유형별로 재구조화 (melt)
df_occur_melted = df_occur.melt(id_vars=['연도'], var_name='범죄유형', value_name='발생건수')
df_arrest_melted = df_arrest.melt(id_vars=['연도'], var_name='범죄유형', value_name='검거건수')

# 두 데이터프레임 병합
df_merged = pd.merge(df_occur_melted, df_arrest_melted, on=['연도', '범죄유형'])

# '발생건수'와 '검거건수'를 숫자형으로 변환 (에러 발생 시 NaN 처리)
df_merged['발생건수'] = pd.to_numeric(df_merged['발생건수'], errors='coerce')
df_merged['검거건수'] = pd.to_numeric(df_merged['검거건수'], errors='coerce')

# 검거율 계산
df_merged['검거율'] = df_merged['검거건수'] / df_merged['발생건수']

# 각 연도별 최대 검거율을 가진 범죄유형 찾기
idx = df_merged.groupby(['연도'])['검거율'].idxmax()
max_arrest_rate_crimes = df_merged.loc[idx]

# 해당 범죄유형들의 검거건수 총합 계산
total_arrest_count = max_arrest_rate_crimes['검거건수'].sum()

# 결과 출력
print(f"각 연도별 최대 검거율을 가진 범죄유형들의 검거건수 총합: {total_arrest_count}")

각 연도별 최대 검거율을 가진 범죄유형들의 검거건수 총합: 9879.0


제 2유형

훈련 데이터로 학습한 모델을 테스트 데이터에 적용하여 예측한 결과를 제출하시오.
(Target: 농업유형(다중분류, {0,1,2}))
% 제출 형식은 ID, pred 두 칼럼만 존재해야 한다.(평가 지표: Marco F1 Score )

제 3유형

1. 모든 칼럼을 활용하여 design을 예측하는 다중회귀 분석을 시행한 후 유의하지 않은 설명변수 개수를 구하시오.(패널티는 포함되지 않으며, 모델의 절편항은 포함)

데이터 분리 조건:

훈련 데이터: id1~id140

테스트 데이터: id141~id200


2. 훈련 데이터(학습용 데이터)의 예측값과 실제값의 피어슨 상관계수를 구하시오.

3. 적합한 모델을 활용하여 테스트 데이터에서의 RMSE를 구하시오.

In [24]:
import pandas as pd
df = pd.read_csv('9_3_1.csv')
#print(df.head())

from statsmodels.formula.api import ols
train = df[df['id'] <= 140]
test = df[df['id'] > 140]
model = ols('design ~ tenure + f2 + f3 + f4 + f5', data=train).fit()

#print(model.summary())
var_count = (model.pvalues[1:] > 0.05).sum()
print(var_count)

from scipy.stats import pearsonr
pred = model.predict(train)
corr, p_val = pearsonr(train['design'], pred)
print(corr)

from sklearn.metrics import mean_absolute_error
import numpy as np
test_pred = model.predict(test)
rmse = np.sqrt(mean_absolute_error(test['design'], test_pred))
print(rmse)

2
0.914750154303963
1.9040871878347285


In [34]:
import pandas as pd
df = pd.read_csv('9_3_2.csv')
#print(df.head())

from statsmodels.formula.api import logit
model = logit('churn ~ col1 + col2 + Phone_Service + Tech_Insurance', data=df).fit()
p_val = model.pvalues['col1']
print(round(p_val, 4))

import numpy as np
odds_ratio = np.exp(model.params['Phone_Service'])
print(round(odds_ratio, 4))

pred = model.predict(df)
count = (pred >=0.3).sum()
print(count)


Optimization terminated successfully.
         Current function value: 0.640721
         Iterations 5
0.0
1.8671
450
